In [12]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [2]:
train = pd.read_csv('csv/train.csv')
train = train.dropna(axis="columns")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4651 entries, 0 to 4650
Data columns (total 10 columns):
title               4651 non-null object
description         4651 non-null object
plot                4651 non-null object
csm_review          4651 non-null object
need_to_know        4651 non-null object
csm_rating          4651 non-null int64
Genre               4651 non-null object
Book type           4651 non-null object
Publication date    4651 non-null object
Last updated        4651 non-null object
dtypes: int64(1), object(9)
memory usage: 363.5+ KB


In [3]:
labelencoder = LabelEncoder()
tokenizer = Tokenizer()
embedding_columns = ['title','description', 'plot', 'csm_review', 'need_to_know']

def label_encoder(df):
    df.loc[:, "genre_cat"] = labelencoder.fit_transform(df.loc[:, "Genre"])
    df.loc[:, "book_type_cat"] = labelencoder.fit_transform(df.loc[:, "Book type"])
    
    return df

def create_embeddings(df):
    for column in embedding_columns:
        tokenizer.fit_on_texts(list(df[column]))
    
    for column in embedding_columns:
        df[column + "_seq"] = tokenizer.texts_to_sequences(df[column])
    
    return df

In [4]:
train = label_encoder(train)
train = create_embeddings(train)

In [5]:
word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))

Found 45360 unique tokens.


In [9]:
X_train = train.loc[:, "genre_cat":"need_to_know_seq"]
y_train = train.loc[:, "csm_rating"]

In [ ]:


gensim_word2vec_tr = GensimWord2VecVectorizer(size=50, min_count=3, sg=1, alpha=0.025, iter=10)
xgb = XGBClassifier(learning_rate=0.01, n_estimators=100, n_jobs=-1)
w2v_xgb = Pipeline([
    ('w2v', gensim_word2vec_tr), 
    ('xgb', xgb)
])
w2v_xgb

In [10]:
xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train, y_train)

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields title_seq, description_seq, plot_seq, csm_review_seq, need_to_know_seq

In [6]:
df_y = df.loc[:, "csm_rating"]

,genre_cat,book_type_cat,title_seq,description_seq,plot_seq,csm_review_seq,need_to_know_seq
0,32,0,"[1, 520, 1377]","[1151, 823, 26455, 17, 116, 1239]","[1088, 4510, 2, 17712, 8850, 33649, 318, 6, 23...","[1, 520, 1377, 37, 23, 541, 434, 1130, 280, 17...","[28, 38, 4, 34, 9, 1, 520, 1377, 7, 3, 611, 23..."
1,12,0,"[285, 12374]","[6678, 29, 5, 1911, 3526, 348, 6, 6364]","[49, 1, 348, 5, 8441, 84, 1803, 90, 78, 14648,...","[16, 226, 91, 48, 143, 3, 21611, 5, 5888, 139,...","[28, 38, 4, 34, 9, 285, 12374, 7, 1656, 21, 19..."
2,16,0,"[1, 55, 11, 94, 2, 329, 18, 347]","[4571, 436, 97, 37, 317, 5, 886, 11, 128, 264]","[35, 119, 83, 1273, 2, 7623, 15, 2140, 209, 27...","[1, 55, 11, 94, 2, 329, 7, 95, 535, 5, 1, 5098...","[28, 38, 4, 34, 9, 1, 55, 11, 94, 2, 329, 7, 3..."
3,38,0,"[3415, 5, 2115, 1, 2166, 4616, 3900, 18, 347]","[58, 540, 11089, 12, 106, 1070, 6701, 5, 1006,...","[179, 6, 7834, 3415, 5, 2115, 568, 3, 937, 90,...","[350, 1006, 50, 215, 230, 82, 3, 5049, 3539, 2...","[28, 38, 4, 34, 9, 3415, 5, 2115, 1, 2166, 461..."
4,23,0,"[4286, 14337, 6821]","[1723, 98, 568, 7568, 2318, 17, 17581, 1807]","[4286, 14337, 6821, 7, 3, 1342, 98, 27, 516, 5...","[827, 270, 2, 13165, 3161, 16, 399, 98, 7, 311...","[28, 38, 4, 34, 9, 4286, 14337, 6821, 21, 1062..."
